In [1]:
%cd ..

/Users/pavanpreetgandhi/git-repos/interpretable-sla


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from source.fit_predict_score_utils import fit_and_predict, build_score_df, calibrate_score_df, compute_metrics

In [3]:
df_original = pd.read_parquet("datasets/1_original_question_set.parquet")
features = df_original.columns[5:]
display(df_original)

,speaker_id,subset,part,part_score,overall_score,pronunciation_and_fluency_1_high,pronunciation_and_fluency_1_medium,pronunciation_and_fluency_1_low,pronunciation_and_fluency_2_high,pronunciation_and_fluency_2_medium,...,discourse_management_1_low,discourse_management_2_high,discourse_management_2_medium,discourse_management_2_low,discourse_management_3_high,discourse_management_3_medium,discourse_management_3_low,discourse_management_4_high,discourse_management_4_medium,discourse_management_4_low
file_id,,,,,,,,,,,,,,,,,,,,,
SI114J-00001-P40018,SI114J-00001,train,4,3.0,NaN,-1.205936,-0.455936,-2.893436,-1.222549,-0.847549,...,-1.537225,-1.540700,-0.540700,-1.665700,-1.470841,-0.595841,-1.595841,-1.338871,-0.588871,-1.776371
SI114J-00002-P10003,SI114J-00002,train,1,4.5,NaN,-0.980234,-0.542734,-3.417734,-0.901788,-0.776788,...,-2.693295,-1.470382,-0.532882,-1.782882,-1.466465,-0.403965,-2.466465,-1.206017,-0.518517,-2.393517
SI114J-00002-P10004,SI114J-00002,train,1,4.5,NaN,-0.989560,-0.552060,-3.177060,-0.862858,-0.925358,...,-2.349973,-1.495469,-0.620469,-1.495469,-1.400662,-0.463162,-2.213162,-1.236796,-0.611796,-1.861796
SI114J-00002-P10005,SI114J-00002,train,1,4.5,NaN,-0.604611,-0.854611,-3.917110,-0.636143,-1.011143,...,-3.560276,-0.997434,-0.622434,-2.497434,-1.213107,-0.525607,-2.338107,-0.816505,-0.691505,-3.066505
SI114J-00002-P10006,SI114J-00002,train,1,4.5,NaN,-0.998290,-0.498290,-4.123290,-0.984130,-0.609130,...,-3.373494,-1.719364,-0.344364,-2.281864,-1.354163,-0.416663,-2.666662,-1.271294,-0.396294,-3.271294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SI137O-01032-P50019,SI137O-01032,eval,5,3.5,3.25,-0.844386,-0.656886,-3.219386,-0.761563,-1.011563,...,-2.540573,-1.300890,-0.550890,-1.988390,-1.490792,-0.490792,-1.928292,-0.937750,-0.687750,-2.375249
SI137O-01032-P50020,SI137O-01032,eval,5,3.5,3.25,-0.739416,-0.739416,-3.364416,-0.838438,-1.025938,...,-2.017626,-1.339611,-0.589611,-1.777111,-1.510109,-0.510109,-1.822609,-0.959150,-0.771650,-1.959151
SI137O-01032-P50021,SI137O-01032,eval,5,3.5,3.25,-0.939854,-0.627354,-2.752354,-0.933854,-0.933854,...,-2.006833,-1.117831,-0.617831,-2.117831,-1.488835,-0.488835,-1.926335,-1.005787,-0.755787,-1.880787


In [4]:
df_exp = df_original.copy()
df_exp[features] = np.exp(df_original[features])
display(df_exp)

,speaker_id,subset,part,part_score,overall_score,pronunciation_and_fluency_1_high,pronunciation_and_fluency_1_medium,pronunciation_and_fluency_1_low,pronunciation_and_fluency_2_high,pronunciation_and_fluency_2_medium,...,discourse_management_1_low,discourse_management_2_high,discourse_management_2_medium,discourse_management_2_low,discourse_management_3_high,discourse_management_3_medium,discourse_management_3_low,discourse_management_4_high,discourse_management_4_medium,discourse_management_4_low
file_id,,,,,,,,,,,,,,,,,,,,,
SI114J-00001-P40018,SI114J-00001,train,4,3.0,NaN,0.299412,0.633854,0.055386,0.294479,0.428464,...,0.214977,0.214231,0.582340,0.189058,0.229732,0.551099,0.202738,0.262142,0.554954,0.169251
SI114J-00002-P10003,SI114J-00002,train,1,4.5,NaN,0.375223,0.581157,0.032787,0.405843,0.459881,...,0.067658,0.229838,0.586911,0.168153,0.230740,0.667667,0.084884,0.299387,0.595403,0.091308
SI114J-00002-P10004,SI114J-00002,train,1,4.5,NaN,0.371740,0.575762,0.041708,0.421955,0.396390,...,0.095372,0.224143,0.537692,0.224143,0.246434,0.629291,0.109354,0.290313,0.542376,0.155393
SI114J-00002-P10005,SI114J-00002,train,1,4.5,NaN,0.546287,0.425449,0.019899,0.529330,0.363803,...,0.028431,0.368825,0.536637,0.082296,0.297272,0.591196,0.096510,0.441974,0.500822,0.046584
SI114J-00002-P10006,SI114J-00002,train,1,4.5,NaN,0.368509,0.607569,0.016191,0.373764,0.543824,...,0.034270,0.179180,0.708671,0.102094,0.258163,0.659243,0.069484,0.280468,0.672809,0.037957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SI137O-01032-P50019,SI137O-01032,eval,5,3.5,3.25,0.429821,0.518464,0.039980,0.466936,0.363650,...,0.078821,0.272289,0.576437,0.136916,0.225194,0.612141,0.145396,0.391508,0.502706,0.092991
SI137O-01032-P50020,SI137O-01032,eval,5,3.5,3.25,0.477393,0.477393,0.034582,0.432385,0.358460,...,0.132971,0.261948,0.554543,0.169126,0.220886,0.600430,0.161604,0.383218,0.462249,0.140978
SI137O-01032-P50021,SI137O-01032,eval,5,3.5,3.25,0.390685,0.534003,0.063778,0.393036,0.393036,...,0.134414,0.326988,0.539113,0.120292,0.225635,0.613341,0.145681,0.365757,0.469641,0.152470


In [5]:
num_options = 3
df_softmax = df_original.copy()
question_options = [features[i:i + num_options] for i in range(0, len(features), num_options)]
for options in question_options:
    exp_values = np.exp(df_original[options])
    df_softmax[options] = exp_values.div(exp_values.sum(axis=1), axis=0)
display(df_softmax)

,speaker_id,subset,part,part_score,overall_score,pronunciation_and_fluency_1_high,pronunciation_and_fluency_1_medium,pronunciation_and_fluency_1_low,pronunciation_and_fluency_2_high,pronunciation_and_fluency_2_medium,...,discourse_management_1_low,discourse_management_2_high,discourse_management_2_medium,discourse_management_2_low,discourse_management_3_high,discourse_management_3_medium,discourse_management_3_low,discourse_management_4_high,discourse_management_4_medium,discourse_management_4_low
file_id,,,,,,,,,,,,,,,,,,,,,
SI114J-00001-P40018,SI114J-00001,train,4,3.0,NaN,0.302848,0.641130,0.056021,0.299627,0.435954,...,0.217355,0.217355,0.590831,0.191815,0.233570,0.560305,0.206125,0.265770,0.562636,0.171594
SI114J-00002-P10003,SI114J-00002,train,1,4.5,NaN,0.379333,0.587522,0.033146,0.413282,0.468310,...,0.068398,0.233361,0.595908,0.170731,0.234661,0.679013,0.086327,0.303608,0.603797,0.092595
SI114J-00002-P10004,SI114J-00002,train,1,4.5,NaN,0.375795,0.582042,0.042163,0.428997,0.403005,...,0.096516,0.227331,0.545338,0.227331,0.250167,0.638823,0.111011,0.293815,0.548918,0.157268
SI114J-00002-P10005,SI114J-00002,train,1,4.5,NaN,0.550896,0.429038,0.020066,0.537327,0.369299,...,0.028661,0.373396,0.543288,0.083316,0.301806,0.600212,0.097982,0.446718,0.506198,0.047084
SI114J-00002-P10006,SI114J-00002,train,1,4.5,NaN,0.371380,0.612302,0.016317,0.378792,0.551139,...,0.034541,0.181000,0.715869,0.103131,0.261593,0.668001,0.070407,0.282949,0.678758,0.038293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SI137O-01032-P50019,SI137O-01032,eval,5,3.5,3.25,0.434925,0.524620,0.040454,0.475409,0.370249,...,0.079798,0.276256,0.584834,0.138910,0.229151,0.622898,0.147951,0.396582,0.509221,0.094197
SI137O-01032-P50020,SI137O-01032,eval,5,3.5,3.25,0.482523,0.482523,0.034954,0.439990,0.364765,...,0.134522,0.265770,0.562636,0.171594,0.224724,0.610864,0.164412,0.388484,0.468601,0.142915
SI137O-01032-P50021,SI137O-01032,eval,5,3.5,3.25,0.395244,0.540234,0.064522,0.399547,0.399547,...,0.135981,0.331499,0.546549,0.121952,0.229151,0.622898,0.147951,0.370249,0.475409,0.154343


In [6]:
num_options = 3
df_one_hot = df_original.copy()
question_options = [features[i:i + num_options] for i in range(0, len(features), num_options)]
for options in question_options:
    max_indices = df_original[options].idxmax(axis=1)
    for option in options:
        df_one_hot[option] = (max_indices == option).astype(int)
display(df_one_hot)

,speaker_id,subset,part,part_score,overall_score,pronunciation_and_fluency_1_high,pronunciation_and_fluency_1_medium,pronunciation_and_fluency_1_low,pronunciation_and_fluency_2_high,pronunciation_and_fluency_2_medium,...,discourse_management_1_low,discourse_management_2_high,discourse_management_2_medium,discourse_management_2_low,discourse_management_3_high,discourse_management_3_medium,discourse_management_3_low,discourse_management_4_high,discourse_management_4_medium,discourse_management_4_low
file_id,,,,,,,,,,,,,,,,,,,,,
SI114J-00001-P40018,SI114J-00001,train,4,3.0,NaN,0,1,0,0,1,...,0,0,1,0,0,1,0,0,1,0
SI114J-00002-P10003,SI114J-00002,train,1,4.5,NaN,0,1,0,0,1,...,0,0,1,0,0,1,0,0,1,0
SI114J-00002-P10004,SI114J-00002,train,1,4.5,NaN,0,1,0,1,0,...,0,0,1,0,0,1,0,0,1,0
SI114J-00002-P10005,SI114J-00002,train,1,4.5,NaN,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
SI114J-00002-P10006,SI114J-00002,train,1,4.5,NaN,0,1,0,0,1,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SI137O-01032-P50019,SI137O-01032,eval,5,3.5,3.25,0,1,0,1,0,...,0,0,1,0,0,1,0,0,1,0
SI137O-01032-P50020,SI137O-01032,eval,5,3.5,3.25,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
SI137O-01032-P50021,SI137O-01032,eval,5,3.5,3.25,0,1,0,1,0,...,0,0,1,0,0,1,0,0,1,0


In [7]:
experiments = [
    {
        "name": "Original Features (logprobs)",
        "dataset": df_original,
        "features": features,
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", LinearRegression())
        ]),
    },
    {
        "name": "Discrete Features",
        "dataset": df_one_hot,
        "features": features,
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", LinearRegression())
        ]),
    },
    {
        "name": "Exponentiated Features (probs)",
        "dataset": df_exp,
        "features": features,
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", LinearRegression())
        ]),
    },
    {
        "name": "Softmax Normalized Features (probs)",
        "dataset": df_softmax,
        "features": features,
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", LinearRegression())
        ]),
    },
]

In [8]:
PARTS = ["part_1", "part_3", "part_4", "part_5", "overall"]
SUBSETS = ["train", "dev", "eval"]
metrics_all = {f"{subset}_{part}": {} for subset in SUBSETS for part in PARTS}
for experiment in experiments:
    print(f"Running experiment: {experiment['name']}")
    predictions, pipelines = fit_and_predict(
        df=experiment["dataset"],
        features=experiment["features"],
        pipeline=experiment["pipeline"],
    )
    score_df = build_score_df(
        df=experiment["dataset"],
        predictions=predictions,
    )
    calibrated_score_df = calibrate_score_df(
        score_df, 
        calibration_set='dev'
    )
    metrics = compute_metrics(
        score_df=calibrated_score_df,
        n_bootstraps=1000,
        subsets=["train", "dev", "eval"],
    )
    for subset in SUBSETS:
        for part in PARTS:
            metrics_all[f"{subset}_{part}"][experiment["name"]] = metrics[subset][part]

Running experiment: Original Features (logprobs)
Running experiment: Discrete Features
Running experiment: Exponentiated Features (probs)
Running experiment: Softmax Normalized Features (probs)


In [9]:
for category, metrics in metrics_all.items():
    print(f"{category}")
    metrics_df = pd.DataFrame(metrics).T.map(lambda x: f"{x[0]:.4f} ± {x[1]:.4f}")
    display(metrics_df)

train_part_1


,rmse,pcc,src,p05,p1
Original Features (logprobs),0.5297 ± 0.0066,0.6965 ± 0.0094,0.6889 ± 0.0101,63.9288 ± 0.8700,94.4345 ± 0.4169
Discrete Features,0.5873 ± 0.0075,0.6138 ± 0.0116,0.6049 ± 0.0123,59.3236 ± 0.8910,91.2050 ± 0.5142
Exponentiated Features (probs),0.5274 ± 0.0066,0.7054 ± 0.0089,0.6978 ± 0.0098,64.6562 ± 0.8623,93.9720 ± 0.4139
Softmax Normalized Features (probs),0.5451 ± 0.0067,0.6801 ± 0.0096,0.6714 ± 0.0103,62.2466 ± 0.8383,93.8474 ± 0.4265


train_part_3


,rmse,pcc,src,p05,p1
Original Features (logprobs),0.5183 ± 0.0068,0.6737 ± 0.0106,0.6540 ± 0.0114,66.2058 ± 0.8779,94.7796 ± 0.3995
Discrete Features,0.6223 ± 0.0077,0.4664 ± 0.0151,0.4300 ± 0.0154,58.9520 ± 0.8932,89.7196 ± 0.5698
Exponentiated Features (probs),0.5173 ± 0.0068,0.6756 ± 0.0104,0.6573 ± 0.0111,67.1350 ± 0.8505,94.8450 ± 0.3947
Softmax Normalized Features (probs),0.5434 ± 0.0072,0.6330 ± 0.0117,0.6235 ± 0.0120,63.8367 ± 0.8991,94.0892 ± 0.4242


train_part_4


...

,rmse,pcc,src,p05,p1
Original Features (logprobs),0.5746 ± 0.0234,0.5983 ± 0.0340,0.5961 ± 0.0388,56.5643 ± 2.8160,92.6227 ± 1.5162
Discrete Features,0.6458 ± 0.0251,0.4372 ± 0.0459,0.3874 ± 0.0507,55.6460 ± 2.8529,87.9923 ± 1.9107
Exponentiated Features (probs),0.5667 ± 0.0232,0.6128 ± 0.0327,0.6004 ± 0.0369,59.0330 ± 2.8400,92.6280 ± 1.5008
Softmax Normalized Features (probs),0.5888 ± 0.0237,0.5704 ± 0.0359,0.5607 ± 0.0411,56.9620 ± 2.8252,92.3167 ± 1.5158


eval_part_5


,rmse,pcc,src,p05,p1
Original Features (logprobs),0.5258 ± 0.0230,0.6920 ± 0.0309,0.6795 ± 0.0346,67.6487 ± 2.7455,93.2993 ± 1.4363
Discrete Features,0.6188 ± 0.0270,0.5339 ± 0.0402,0.5198 ± 0.0413,55.9923 ± 2.7639,91.9853 ± 1.5569
Exponentiated Features (probs),0.5279 ± 0.0235,0.6908 ± 0.0305,0.6819 ± 0.0344,67.6283 ± 2.7847,93.6360 ± 1.3739
Softmax Normalized Features (probs),0.5372 ± 0.0236,0.6726 ± 0.0320,0.6544 ± 0.0358,66.6787 ± 2.8309,93.6317 ± 1.4165


eval_overall


,rmse,pcc,src,p05,p1
Original Features (logprobs),0.4049 ± 0.0162,0.7758 ± 0.0200,0.7753 ± 0.0243,78.6370 ± 2.2622,98.9823 ± 0.5729
Discrete Features,0.4721 ± 0.0191,0.6826 ± 0.0295,0.6649 ± 0.0352,70.3150 ± 2.5775,96.6680 ± 1.0169
Exponentiated Features (probs),0.4037 ± 0.0156,0.7773 ± 0.0186,0.7828 ± 0.0224,77.8770 ± 2.2964,98.9597 ± 0.5665
Softmax Normalized Features (probs),0.4188 ± 0.0165,0.7587 ± 0.0210,0.7670 ± 0.0247,77.6513 ± 2.4006,98.6333 ± 0.6653
